# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import pickle
import ast

from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score
from hyperopt import hp
import lightgbm as lgb
from skopt import BayesSearchCV

sys.path += ['./ilikeds']
import eda 
import helper_functions as h
import train_classifier as t

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
# azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
# customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

azdias = pd.read_pickle ('../data/azdias.p')    
customers = pd.read_pickle ('../data/customers.p')    

In [ ]:
# read in feature info file
feat_info = pd.read_csv('./feats_info.csv', sep=';', names=['feat', 'type', 'unknown'])
feat_info.set_index('feat', inplace =True)

In [ ]:
# create  a EDA  instance for Azdias.
eda_azdias= eda.EDA(azdias, feat_info, label = 'Azdias')

# create  a EDA  instance for Azdias. for  customers
eda_customers= eda.EDA(customers, feat_info, label = 'Customers')

### Data Cleaning

#### Step 1: Removing undefined features

In [ ]:
feats_todrop = []
feats_wo_def = list(set(eda_azdias.data.columns) - set(eda_azdias.feat_info.index))

eda_azdias.data.drop(columns = feats_wo_def, inplace =True)
eda_customers.data.drop(columns = feats_wo_def, inplace =True)

# Removing the three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP') in Customers
feats_customers_excl  = list(set(eda_customers.data.columns) - set(eda_azdias.data.columns))
feats_customers_excl
eda_customers.data.drop(columns = feats_customers_excl, inplace =True)

####  Step 2:  Converting of missing and unknown data to NaN

In [ ]:
eda_azdias.missing2nan()
eda_customers.missing2nan()

####  Step 3: Handling missing values

##### a. Handling missing values for rows

In [ ]:
rows_n_nans = azdias.isnull().sum(axis=1)
plt.hist(rows_n_nans / azdias.shape[1], bins=90)

In [ ]:
_, rows_droped = h.split_dataset(azdias, threshold=0.25)
n_deleted_rows = rows_droped.shape[0]
print(f'Before delete the missing rows, {eda_azdias} has {eda_azdias.data.shape[0]} rows')
azdias.drop(index = rows_droped.index, inplace =True)
print(f'After delete the missing rows, {eda_azdias} has {eda_azdias.data.shape[0]} rows')
print(f'Delete {n_deleted_rows} rows in total')

In [ ]:
_, rows_droped = h.split_dataset(customers, threshold=0.25)
n_deleted_rows = rows_droped.shape[0]
print(f'Before delete the high missing rate rows, {eda_customers} has {eda_customers.data.shape[0]} rows')
eda_customers.data.drop(index = rows_droped.index, inplace =True)
print(f'After delete the hitg missing rate rows, {eda_customers} has {eda_customers.data.shape[0]} rows')
print(f'Delete {n_deleted_rows} rows in total')

##### b. Handling missing values for columns

In [ ]:
eda_azdias.feat_info.percent_of_nans.sort_values().hist(bins = 40, alpha = 0.7)
plt.xlabel('Missing Rate ')
plt.ylabel('Num Of Features')
plt.title('Distribution of missing value per column')

In [ ]:
thr_col_missing = 0.37
feats_todrop_azdias = eda_azdias.feat_info.loc[eda_azdias.feat_info.percent_of_nans > thr_col_missing].index
feats_todrop_azdias

In [ ]:
feats_todrop_customers = eda_customers.feat_info.loc[eda_customers.feat_info.percent_of_nans >thr_col_missing ].index
feats_todrop_customers

In [ ]:
# Get features with high missing rates in both datasets
feats_todrop_comm = set(feats_todrop_customers).intersection(set(feats_todrop_azdias))
feats_todrop_comm = list(feats_todrop_comm)
feats_todrop_comm

In [ ]:
print(f'Before delete the high missing rate features/columns, {eda_azdias} has {eda_azdias.data.shape[1]} features.')
azdias.drop(columns=feats_todrop_comm, inplace=True)
print(f'After delete the high missing rate features/columns, {eda_azdias} has {eda_azdias.data.shape[1]} features.')

print()

print(f'Before delete the high missing rate features/columns, {eda_customers} has {eda_customers.data.shape[1]} features.')
customers.drop(columns=feats_todrop_comm, inplace=True)
print(f'After delete the high missing rate features/columns, {eda_customers} has {eda_customers.data.shape[1]} features.')

print()
print('Delete {len(feats_todrop_comm)} features in total')

####  Step 4:  Scrub for duplicate features

In [ ]:
feats_fein =[x for x in eda_azdias.data.columns if x.endswith( '_FEIN')]
feats_grob = [x for x in eda_azdias.data.columns if x.endswith( '_GROB')]
feats_duplicate = [x for x in zip(pd.Series(feats_grob).sort_values(), pd.Series(feats_fein).sort_values())]
h.plot_2feats_comparison(eda_azdias.data, feats_duplicate)   

Both features on the same line have the same meaning. Features with a suffix of _GROB on the left represent coarse-grained information, and _FEIN represents fine-grained features. I decided to delete these duplicate features, 'ALTERSKATEGORIE_FEIN', 'LP_FAMILIE_GROB', 'LP_LEBENSPHASE_FEIN', 'LP_STATUS_GROB', 'CAMEO_DEU_2015'. Because some of them are undefined, or  use the corresponding _GROB feature is enough.

In [ ]:
feats_duplicated = ['ALTERSKATEGORIE_FEIN', 'LP_FAMILIE_GROB', 'LP_LEBENSPHASE_FEIN', 'LP_STATUS_GROB','CAMEO_DEU_2015']

In [ ]:
#  ANZ_STATISTISCHE_HAUSHALTE, ANZ_HAUSHALTE_AKTIV :  number of households in the building
#  Delete ANZ_STATISTISCHE_HAUSHALTE, because they are duplicated.

feats_HAUSHALTE = ['ANZ_STATISTISCHE_HAUSHALTE', 'ANZ_HAUSHALTE_AKTIV']
h.plot_boxplot(eda_azdias.data, feats_HAUSHALTE, n_cols =2,  figsize = (16, 4))     

In [ ]:
feats_duplicated.extend(['ANZ_STATISTISCHE_HAUSHALTE'])
azdias.drop(columns=feats_duplicated, inplace=True)
customers.drop(columns=feats_duplicated, inplace=True)

#### Step 5: Re-encodings features

-  'CAMEO_DEUG_2015' 
- 'CAMEO_INTL_2015' :“New German CAMEO Typology established together with Call Credit in late 2015”.
 - 'EINGEFUEGT_AM',  
 -'OST_WEST_KZ' 

In [ ]:
encoding = [
                  'OST_WEST_KZ',
                  'CAMEO_DEUG_2015',
                  'CAMEO_INTL_2015',
                    'CAMEO_INTL_2015',
                  'EINGEFUEGT_AM']

# h.check_features(eda_azdias, encoding)     
for x in encoding:
     print(x, eda_azdias.data[x].unique())

In [ ]:
feats_encoding = ['OST_WEST_KZ','CAMEO_DEUG_2015','CAMEO_INTL_2015','EINGEFUEGT_AM']

for x in feats_encoding:
    eda_azdias.re_encoding(x)
    eda_customers.re_encoding(x)        

####  Step 6. Split Mixed Features

In [ ]:
mixed = eda_azdias.feat_info[feat_info['type'] == 'mixed']
mixed

In [ ]:
mixed_feats = ['CAMEO_INTL_2015', 'LP_LEBENSPHASE_GROB', 'PRAEGENDE_JUGENDJAHRE']
for x in mixed_feats:
    eda_azdias.process_mixed_feat(x)
    eda_customers.process_mixed_feat(x)    

In [ ]:
feats_splited = ['CAMEO_INTL_2015_SPLIT_WEALTH', 'CAMEO_INTL_2015_SPLIT_LIFE_STAGE','LP_LEBENSPHASE_GROB_SPLIT_FAMILY','LP_LEBENSPHASE_GROB_SPLIT_AGE','LP_LEBENSPHASE_GROB_SPLIT_INCOME','PRAEGENDE_JUGENDJAHRE_SPLIT_DECADE','PRAEGENDE_JUGENDJAHRE_SPLIT_MOVEMENT']
eda_azdias.data[feats_splited]

In [ ]:
feat_info_split = eda_azdias.build_feat_info(feats_splited)
eda_azdias.feat_info= pd.concat([eda_azdias.feat_info, feat_info_split], sort = False)

feat_info_split = eda_customers.build_feat_info(feats_splited)
eda_customers.feat_info= pd.concat([eda_customers.feat_info, feat_info_split], sort = False)
eda_azdias.feat_info.loc[feats_splited]

In [ ]:
eda_azdias.collecting_stats()
eda_customers.collecting_stats()

#### Step 7.  Cleaning outliers

In [ ]:
eda_azdias.feat_info

#####  a. Analysis outliers for  numeric features

In [ ]:
eda_azdias.feat_info.loc[eda_azdias.feat_info.type == 'numeric']

In [ ]:
eda_azdias.feat_info[eda_azdias.feat_info.value_count > 100]

In [ ]:
# Delete  'KBA13_ANZAHL_PKW'  and ' ANZ_HAUSHALTE_AKTIV'  , Because they have 221 and 1245 unique values, respectively, it is not good for subsequent cluster comparisons
feats_todrop.extend(['KBA13_ANZAHL_PKW','ANZ_HAUSHALTE_AKTIV'])

In [ ]:
numeric_outlier_feats = ['MIN_GEBAEUDEJAHR' , 'EINGEZOGENAM_HH_JAHR']
h.plot_boxplot(eda_azdias.data, numeric_outlier_feats, n_cols=2,  figsize = (16, 4))

In [ ]:
eda_azdias.clean_outlier(numeric_outlier_feats)
h.plot_boxplot(eda_azdias.data, numeric_outlier_feats, n_cols=2,  figsize = (16, 4))

In [ ]:
h.plot_boxplot(eda_customers.data, numeric_outlier_feats, n_cols=2,  figsize = (16, 4))     

#####  a. Analysis outliers for  categorical features

In [ ]:
eda_azdias.feat_info.loc[eda_azdias.feat_info.type == 'categorical']

In [ ]:
h.check_features(eda_azdias, 'categorical')      

In [ ]:
cat_outlier_feats  = ['D19_KONSUMTYP', 'EINGEZOGENAM_HH_JAHR']
h.plot_boxplot(eda_azdias.data, cat_outlier_feats, n_cols=2 ,  figsize = (16, 4))     

In [ ]:
# eda_azdias.clean_outlier(numeric_outlier_feats)
# h.plot_boxplot(eda_azdias.data, cat_outlier_feats, n_cols=2 ,  figsize = (16, 4))     

####  Step  8:  Impute missing value

In [ ]:
imputer = Imputer(strategy='median')
eda_azdias.data_imputed = pd.DataFrame(imputer.fit_transform(eda_azdias.data))
eda_customers.data_imputed = pd.DataFrame(imputer.fit_transform(eda_customers.data))

####  Step  9:   Feature  Scaling

In [ ]:
eda_azdias.data_scaled = StandardScaler().fit_transform(eda_azdias.data_imputed)    
eda_customers.data_scaled = StandardScaler().fit_transform(eda_customers.data_imputed)   

In [ ]:
# Save the droped  features into a file.
for x in (feats_todrop_comm, feats_duplicated, feats_wo_def):
    feats_todrop.extend(x)

pd.Series(feats_todrop).sort_values().to_csv('feats_dropped.csv', index=False)

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

###   Data comparison   Customers vs. Azdias

#### D19_  Features Comparison

D19_ features is about consumer mail order activities 

In [ ]:
comm_feats = set(eda_customers.data.columns).intersection(set(eda_azdias.data.columns))
feats_D19 =set([x for x in comm_feats if x.startswith( 'D19_')])
feats_D19 = pd.Series(list(feats_D19)).sort_values().tolist()

h.plot_feats_comparison(eda_azdias.data, eda_customers.data, feats_D19)

The feature comparison in the above figure shows:
- D19_SOZIALES: value 1 is significantly higher in customers than in azdias. Although the feature dictionary does not define D19_SOZIALES. But I guess it represents a commodity. 
- D19_KONSUMTYP:  The values of 2, 3 and 9, customers were significantly higher than azdias, They are 2: versatile, 3: Gourmet, 9 inactive, while value 9 is significantly lower than azdias. The target customers are gourmet, versatile and active.
- D19_KONSUMTYP_MAX:  value 2 is significantly higher than azdias, while value 8 is  significantly lower than azdias.

#### CAMEO_  Features Comparison

CAMEO_DEUG_2015:  New German CAMEO Typology established together with Call Credit in late 2015

In [ ]:
h.plot_feats_comparison(eda_azdias.data, eda_customers.data, ['CAMEO_DEUG_2015'], fig_height=5, fig_aspect=1.6)

- CAMEO_DEUG_2015:  value 1, 2 is significantly higher in customers than in azdias. while  value 9 is significantly lower than azdias. Indicates that the target customers are middle and upper class people.

#### CJT_ Features Comparison

CJT_ features is about mail order activities of consumers

In [ ]:
feats_CJT =set([x for x in eda_customers.data.columns if x.startswith( 'CJT_')])
feats_CJT = pd.Series(list(feats_CJT)).sort_values().tolist()

h.plot_feats_comparison(eda_azdias.data, eda_customers.data, feats_CJT)

- CJT_GESAMTTYP :   customer journey typology.   value 2  is significantly higher in customers than in azdias. Indicates that the target customers are advertising- and consumptiontraditionalist

### Principal component analysis (PCA)

In [ ]:
h.do_pca(eda_azdias, 200)

print(f'total explained_variance:  {eda_azdias.pca.explained_variance_ratio_.sum()}')
# print('explained_variance_ratio: ', pca.explained_variance_ratio_)
# print('explained_variance: ', pca.explained_variance_)
print('n_components: ', eda_azdias.pca.n_components_)

h.scree_plot(eda_azdias)

In [ ]:
h.do_pca(eda_customers, 200)

print(f'total explained_variance:  {eda_customers.pca.explained_variance_ratio_.sum()}')
# print('explained_variance_ratio: ', pca.explained_variance_ratio_)
# print('explained_variance: ', pca.explained_variance_)
print('n_components: ', eda_customers.pca.n_components_)

h.scree_plot(eda_customers)

### Clustering with KMean

In [ ]:
h.do_pca(eda_azdias, 100)
h.do_pca(eda_customers, 100)

In [ ]:
scores = []
centers = list(range(1,21))

for center in centers:
    _, score = h.get_kmeans_score(eda_customers.X_pca, center)
    scores.append(score)
    
plt.plot(centers, scores, linestyle='--', marker='o', color='b')
plt.xlabel('K')
plt.ylabel('SSE')
plt.title('SSE vs. K')

####  Clustering Comparison  Azdias vs Customers 

In [ ]:
model_c, score = h.get_kmeans_score(eda_customers.X_pca, 10)
print(score)
model_a, score = h.get_kmeans_score(eda_azdias.X_pca, 10)
print(score)

In [ ]:
preds_c = model_c.predict(eda_customers.X_pca)
preds_a = model_a.predict(eda_azdias.X_pca)

counts_c, counts_a = h.plot_cluster_comparison(preds_c, preds_a)

In [ ]:
comp_diff_s = (counts_c.percent - counts_a.percent).sort_values(ascending=False)
pd.DataFrame(
    {
        'cluster': comp_diff_s.index,
        'diff_pct': comp_diff_s.values
    }
)

In [ ]:
h.list_component(eda_customers, 2)

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
# mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')
mailout_train = pd.read_pickle ('..//data//mailout_train.p')    
mailout_train.shape

In [ ]:
positive_cnts = mailout_train[mailout_train['RESPONSE'] == 1].shape[0]
total_cnts = mailout_train.shape[0]
 
print(f'The train set contains only {positive_cnts / total_cnts *100 :1.2f}% customers with positive response')

In [ ]:
# extract RESPONSE column
response = mailout_train['RESPONSE']

# drop RESPONSE column
mailout_train.drop(labels=['RESPONSE'], axis=1, inplace=True)

# find features to drop because of many missing values
missing_per_column = mailout_train.isnull().mean()

plt.hist(missing_per_column, bins=34)

### Data Cleaning

In [ ]:
# read in feature info file
feat_info = pd.read_csv('./feats_info.csv', sep=';', names=['feat', 'type', 'unknown'])
feat_info.set_index('feat', inplace =True)

eda_mailout_train = eda.EDA(mailout_train, feat_info, label = 'mailout_train')
# eda_mailout_train.data

In [ ]:
eda_mailout_train.data_pipeline()

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
# mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')